In [1]:
!unzip /content/drive/MyDrive/eye_gender_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: eye_gender_data/train/Image_4799.jpg  
  inflating: eye_gender_data/train/Image_48.jpg  
  inflating: eye_gender_data/train/Image_480.jpg  
  inflating: eye_gender_data/train/Image_4800.jpg  
  inflating: eye_gender_data/train/Image_4801.jpg  
  inflating: eye_gender_data/train/Image_4802.jpg  
  inflating: eye_gender_data/train/Image_4803.jpg  
  inflating: eye_gender_data/train/Image_4804.jpg  
  inflating: eye_gender_data/train/Image_4805.jpg  
  inflating: eye_gender_data/train/Image_4806.jpg  
  inflating: eye_gender_data/train/Image_4807.jpg  
  inflating: eye_gender_data/train/Image_4808.jpg  
  inflating: eye_gender_data/train/Image_4809.jpg  
  inflating: eye_gender_data/train/Image_481.jpg  
  inflating: eye_gender_data/train/Image_4810.jpg  
  inflating: eye_gender_data/train/Image_4811.jpg  
  inflating: eye_gender_data/train/Image_4812.jpg  
  inflating: eye_gender_data/train/Image_4813.jpg  
  inflating: eye_

In [2]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set

In [3]:
labels = pd.read_csv("/content/eye_gender_data/Training_set.csv") # loading the labels
labels.head() # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,male
1,Image_2.jpg,female
2,Image_3.jpg,female
3,Image_4.jpg,female
4,Image_5.jpg,male


In [4]:
labels.tail()

,filename,label
9215,Image_9216.jpg,male
9216,Image_9217.jpg,male
9217,Image_9218.jpg,male
9218,Image_9219.jpg,male
9219,Image_9220.jpg,female


In [5]:
file_paths = [[fname, '/content/eye_gender_data/train/' + fname] for fname in labels['filename']]

In [6]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
  print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
  print('Number of labels does not match the number of filenames')

Number of labels i.e.  9220 matches the number of filenames i.e.  9220


In [7]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/eye_gender_data/train/Image_1.jpg
1,Image_2.jpg,/content/eye_gender_data/train/Image_2.jpg
2,Image_3.jpg,/content/eye_gender_data/train/Image_3.jpg
3,Image_4.jpg,/content/eye_gender_data/train/Image_4.jpg
4,Image_5.jpg,/content/eye_gender_data/train/Image_5.jpg


In [8]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()

,filename,filepaths,label
0,Image_1.jpg,/content/eye_gender_data/train/Image_1.jpg,male
1,Image_2.jpg,/content/eye_gender_data/train/Image_2.jpg,female
2,Image_3.jpg,/content/eye_gender_data/train/Image_3.jpg,female
3,Image_4.jpg,/content/eye_gender_data/train/Image_4.jpg,female
4,Image_5.jpg,/content/eye_gender_data/train/Image_5.jpg,male


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

In [16]:
data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):

  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
  data.append([new_img_array, train_data['label'][i]])


In [17]:
data[5]

[array([[207, 207, 209, ..., 188, 189, 189],
        [207, 207, 208, ..., 189, 189, 188],
        [206, 207, 208, ..., 192, 189, 187],
        ...,
        [184, 184, 184, ..., 182, 183, 183],
        [184, 184, 185, ..., 181, 180, 179],
        [184, 184, 185, ..., 181, 178, 177]], dtype=uint8), 1]

In [18]:
np.random.shuffle(data)

In [19]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [20]:
np.unique(y, return_counts=True)

(array([0, 1]), array([4162, 5058]))

In [21]:
x = x.reshape(-1, 100, 100, 1)

In [22]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)



In [30]:
cnn = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),

tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
tf.keras.layers.MaxPooling2D((2, 2)),

# tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dense(131, activation='sigmoid')
])

In [37]:
cnn.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

In [40]:
cnn.fit(X_train, y_train, epochs=10, batch_size=1)

Epoch 1/10
6454/6454 [==============================] - 176s 27ms/step - loss: 0.4658 - accuracy: 0.7784
Epoch 2/10
6454/6454 [==============================] - 179s 28ms/step - loss: 0.3512 - accuracy: 0.8531
Epoch 3/10
6454/6454 [==============================] - 180s 28ms/step - loss: 0.2776 - accuracy: 0.8881
Epoch 4/10
6454/6454 [==============================] - 177s 27ms/step - loss: 0.2882 - accuracy: 0.8776
Epoch 5/10
6454/6454 [==============================] - 178s 28ms/step - loss: 0.2295 - accuracy: 0.9072
Epoch 6/10
6454/6454 [==============================] - 181s 28ms/step - loss: 0.3689 - accuracy: 0.8065
Epoch 7/10
6454/6454 [==============================] - 182s 28ms/step - loss: 0.2368 - accuracy: 0.9224
Epoch 8/10
6454/6454 [==============================] - 186s 29ms/step - loss: 0.1651 - accuracy: 0.9368
Epoch 9/10
6454/6454 [==============================] - 189s 29ms/step - loss: 0.1452 - accuracy: 0.9464
Epoch 10/10
6454/6454 [==============================] 

In [41]:
cnn.evaluate(X_val, y_val)

87/87 [==============================] - 9s 100ms/step - loss: 0.5604 - accuracy: 0.8547


[0.5604400038719177, 0.8546637892723083]

In [42]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/eye_gender_data/Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [43]:
file_paths = [[fname, '/content/eye_gender_data/test/' + fname] for fname in test_image_order['filename']]

In [44]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
  print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
  print('Number of image names does not match the number of filepaths')

Number of image names i.e.  2305 matches the number of file paths i.e.  2305


In [45]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/eye_gender_data/test/Image_1.jpg
1,Image_2.jpg,/content/eye_gender_data/test/Image_2.jpg
2,Image_3.jpg,/content/eye_gender_data/test/Image_3.jpg
3,Image_4.jpg,/content/eye_gender_data/test/Image_4.jpg
4,Image_5.jpg,/content/eye_gender_data/test/Image_5.jpg


In [47]:
test_pixel_data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):

  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array

  test_pixel_data.append(new_img_array)

In [48]:
test_pixel_data = np.array(test_pixel_data)

In [49]:
test_pixel_data = test_pixel_data.reshape(-1, 100, 100, 1)

In [50]:
pred = cnn.predict(test_pixel_data)

In [51]:
# The predicted values are the probabilities value
pred[0]

array([6.9019198e-04, 9.9999857e-01, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [52]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [53]:
predictions = le.inverse_transform(prediction)

In [55]:
res = pd.DataFrame({ 'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)

# To download the csv file locally
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>